In [2]:
import sys
import os

project_root = os.path.abspath("..")
sys.path.append(project_root)

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from config import *
from Data.raw_data_loader import load_raw_crypto_csv
from Models.pair_trading import build_spread, pair_signal_zscore
from Models.ou import ou_signal_on_spread
from Backtest.engine import run_backtest
from Backtest.metrics import build_summary_table


In [3]:
df = load_raw_crypto_csv(DATA_PATH)

df = df.loc["2017-11-12":].copy()
btc = df["BTC-USD_close"].astype(float)
eth = df["ETH-USD_close"].astype(float)

mask = (btc > 0) & (eth > 0)
mask &= (btc.pct_change().abs() < 1.0)
mask &= (eth.pct_change().abs() < 1.0)
df = df.loc[mask].copy()

spread = build_spread(df, window=PAIR_BETA_WINDOW)

print("Data length:", len(df))
print("Spread length:", len(spread))


Data length: 3010


In [5]:
from statsmodels.tsa.stattools import adfuller

adf_stat, pvalue, *_ = adfuller(spread.dropna())

print("ADF stat:", adf_stat)
print("p-value:", pvalue)

ADF stat: -0.9523989641796868
p-value: 0.7702276016236705


In [ ]:
z_pos = pair_signal_zscore(
    spread,
    window=60,
    entry_z=2.0,
    exit_z=0.5,
)

z_results = run_backtest(
    price_series=spread,
    position=z_pos,
    fee_bps=FEE_BPS,
    slippage_bps=SLIPPAGE_BPS,
    leverage_cap=LEVERAGE_CAP,
)


In [ ]:
ou_pos = ou_signal_on_spread(
    spread,
    window=90,
    entry_z=1.5,
    exit_z=0.3
)

ou_results = run_backtest(
    price_series=spread,
    position=ou_pos,
    fee_bps=FEE_BPS,
    slippage_bps=SLIPPAGE_BPS,
    leverage_cap=LEVERAGE_CAP,
)


In [ ]:
summary = build_summary_table({
    "Z-Score Pair": {
        "returns": z_results["net_returns"],
        "position": z_pos,
    },
    "OU Pair": {
        "returns": ou_results["net_returns"],
        "position": ou_pos,
    }
})

summary